# <center><strong>Natural Language Processing and Text Mining (NLPTM)<br> Unstructured Data Analysis (UDA)*</strong></center>
    
## <center>(C) Taufik Sutanto - 2020 <br><strong><font color="blue"> tau-data Indonesia ~ https://tau-data.id</font></strong></center>

## Outline Lesson NLPTM-01 :

* Pendahuluan
* Review String di Python
* Filtering (stopwords)
* Replace slang/typos/singkatan
* Spell Check
* Machine translation
* Pengenalan Reguler expression 
* Encodings

# Modul-modul yang digunakan di Lesson ini 
## Silahkan install melalui terminal jika dijalankan secara lokal (PC/Laptop)

### Perhatian: Anda harus menjalankan setiap cell secara berurutan dari paling atas, tanpa terlewat satu cell-pun.

In [ ]:
"""
Installing Modules & getting the necessary files for "Google Colab"
Jika dijalankan di komputer lokal (PC/Laptop) silahkan unduh secara manual dan 
lakukan instalasi module di terminal/command prompt 
"""
import nltk
!wget -P data/ https://github.com/taudata-indonesia/eLearning/blob/master/data/slang.txt
!wget -P data/ https://github.com/taudata-indonesia/eLearning/blob/master/data/stopwords_id.txt
!wget -P data/ https://github.com/taudata-indonesia/eLearning/blob/master/data/stopwords_en.txt
!wget -P data/ https://github.com/taudata-indonesia/eLearning/blob/master/data/corpus_sederhana.txt
!wget -P / https://github.com/taudata-indonesia/eLearning/blob/master/lib/taudataNlpTm.py
!pip install unidecode textblob sastrawi
nltk.download('popular')

## Review Tipe Variabel di Python

In [ ]:
a = 2.3 # Floating Point (tidak sama dengan bilangan Real)
b = 3.0   # Integer
c = True # T/F Boolean
d = 'python' # String
e = [a,b,c,d] # List
f = (a,b,c,d) # Tuple
g = set([a,b,c,d]) # Set
h = {'a':1, 'b':2, 7:'abc'} # Dictionary : keys, values, items

In [ ]:
# Sehingga kita bisa melakukan preprocessing dasar dengan string manipulation seperti berikut:
S = 'coba '
print(S)
print(S.strip())
print(S.replace("o","i"))
print(S.isalnum())
print(S.strip().isalnum()) # dsb

In [ ]:
X = '84'
X.isdigit() 

In [ ]:
help(X.isdigit)

In [ ]:
# String adalah Tuple, sehingga
S1 = 'apa '
S2 = 'kabar'
S1+S2

In [ ]:
S2 = 'abcdefghijklmnopqrst'
S2[9:]

In [ ]:
# String adalah Tuple sehingga bisa di akses seperti List
S = 'NLP dan TextMining di Python'
print('7 character pertama: ', S[:7])
print('7 character terakhir: ', S[-7:])

In [ ]:
# Tapi hati-hati karena ia tuple maka:
try:
    S[5] = 'o'
except:
    print('Error, tuple tidak bisa dirubah nilainya (inplace)')

In [ ]:
# Loading Stopwords: Ada beberapa cara
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()

NLTK_StopWords = stopwords.words('english')
Sastrawi_StopWords_id = factory.get_stop_words()

df=open('data/stopwords_en.txt',"r",encoding="utf-8", errors='replace')
en_stop = df.readlines()
df.close()
en_stop = [t.strip().lower() for t in en_stop]

df=open('data/stopwords_id.txt',"r",encoding="utf-8", errors='replace')
id_stop = df.readlines()
df.close()
id_stop = [t.strip().lower() for t in id_stop]

N = 10
print(NLTK_StopWords[:N])
print(Sastrawi_StopWords_id[:N])
print(en_stop[:N])
print(id_stop[:N])
print(len(Sastrawi_StopWords_id), len(id_stop), len(NLTK_StopWords), len(en_stop))

### Diskusi: Apakah sebaiknya kita menggunakan daftar stopwords bawaan modul atau custom milik kita sendiri?

In [ ]:
# Tipe variabel memiliki aplikasi optimal yang berbeda-beda, misal
L = list(range(10**7))
S = set(range(10**7)) # selain unik dan tidak memiliki keterurutan, set memiliki fungsi lain.

In [ ]:
%%timeit
9000000 in L

In [ ]:
%%timeit
9000000 in S

In [ ]:
# Tips: selalu rubah list stopwords ke bentuk set, karena di Python jauh lebih cepat untuk cek existence di set ketimbang list
NLTK_StopWords = set(NLTK_StopWords)
Sastrawi_StopWords_id = set(Sastrawi_StopWords_id)
en_stop = set(en_stop)
id_stop = set(id_stop)

In [ ]:
St = ['ada', 'pesawat']
'ada' in St

In [ ]:
# Cara menggunakan stopwords
from textblob import TextBlob

T = "I am doing NLP at tau-data Indonesia,... \
    adapun saya anu sedang belajar NLP di tau-data Indonesia"
T = T.lower()
id_stop.add('anu')
Tokens = TextBlob(T).words # Tokenisasi 
T2 = [t for t in Tokens if t not in id_stop] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
T2 = [t for t in T2 if t not in en_stop] # Sastrawi_StopWords_id Personal_StopWords_en Personal_StopWords_id
print(' '.join(T2))
# Catatan: Selalu lakukan Stopword filtering setelah tokenisasi (dan normalisasi).

## Menangani Slang atau Singkatan di Data Teks 

In [ ]:
# Sebuah contoh sederhana 
T = 'jangan ragu gan, langsung saja di order pajangannya.'
# Misal kita hendak mengganti setiap singkatan (slang) dengan bentuk penuhnya. 
# Dalam hal ini kita hendak mengganti 'gan' dengan 'juragan'
H = T.replace('gan','juragan')
print(H)
# Kita tidak bisa melakukan ini

In [ ]:
D = {'yg':'yang', 'gan':'juragan'}
D['gan']

In [ ]:
A = [2,5,1,3]

for i, a in enumerate(A):
    print(a, i)

In [ ]:
# dengan tokenisasi
slangs = {'gan':'juragan', 'yg':'yang', 'dgn':'dengan'} #dictionary sederhana berisi daftar singkatan dan kepanjangannya

T = 'jangan ragu gan, langsung saja di order pajangan yg diatas.'
T = TextBlob(T).words
T

In [ ]:
for i,t in enumerate(T):
    if t in slangs.keys():
        T[i] = slangs[t]
print(' '.join(T))

In [ ]:
# Loading Slang dan Singkatan dari File
# Contoh memuat word fix melalui import file. 
df=open('data/slang.txt',"r",encoding="utf-8", errors='replace')
slangS = df.readlines(); df.close()
slangS[:5] 

In [ ]:
slangS = [t.strip('\n').strip() for t in slangS]
print(slangS[:5])

In [ ]:
A = 'luv:love'
B = A.split(':')
B

In [ ]:
A ='  apa  '
A.strip()

In [ ]:
# pisahkan berdasarkan ':'
slangS = [t.split(":") for t in slangS]
slangS = [[k.strip(), v.strip()] for k,v in slangS]
print(slangS[:3])
slangS = {k:v for k,v in slangS}
print(slangS['7an'])

In [ ]:
# Test it!
tweet = 'I luv u say. serius gan!, tapi ndak tau kalau sesok.'
T = TextBlob(tweet).words

for i,t in enumerate(T):
    if t in slangS.keys():
        T[i] = slangS[t]
        
print(' '.join(T))

<h3>Tujuan Spellcheck:</h3>

<ol>
	<li>	Cleaning Data	</li>
	<li>Word suggestions</li>
	<li>OCR/hand writing (Image) recognition</li>
	<li>Speech Recognition</li>
	<li>Machine Translation</li>
</ol>


In [ ]:
# aplikasi spell check di textBlob
from textblob import Word

w = Word('industri')
w.spellcheck()

In [ ]:
w = Word('jang4n')
w.spellcheck()
# Kendalanya kalau Bahasa Indonesia ==> perlu pendekatan umum

In [ ]:
import taudataNlpTm as tau
# http://norvig.com/spell-correct.html
# corpus = 'data/kata_dasar.txt'
print(tau.correction('benul'))

In [ ]:
kombinasi = tau.edits1('benul')
list(kombinasi)[:5]

In [ ]:
tau.known(tau.edits1('benul'))

In [ ]:
print(tau.known(tau.edits2('benul')))

In [ ]:
print(tau.WORDS.most_common(10))

In [ ]:
#Language Detection (TextBlob)
from textblob import TextBlob
T = "Aku ingin mengerti NLP dalam bahasa Inggris"
U = "jarene iki boso jowo"
print(TextBlob(T).detect_language())
print(TextBlob(U).detect_language())

In [ ]:
# Machine Translation (TextBlob)
# Butuh koneksi internet, limited calls. Error otherwise. Need "try" and "catch".
T = "Aku ingin mengerti NLP dalam bahasa Inggris. I love you"
print(TextBlob(T).translate(to='en'))
print(TextBlob(T).translate(to='ar-sa'))
print(TextBlob(T).translate(to='ja'))
# daftar kode bahasa : http://www.cardinalpath.com/resources/tools/google-analytics-language-codes/
# Perhatikan TextBlob secara automatis akan mendeteksi bahasa asal

In [ ]:
# Kalau secara spesifik ingin translate dari suatu bahasa ke bahasa lain:
T = "Aku hanya ingin mengatakan ... saya lapar ... hungry bro"
print(TextBlob(T).translate(from_lang ='id', to='en'))

<p><strong>Beberapa Reguler Expression yang sering digunakan di NLP/Text Mining</strong></p>

<ol>
	<li>Menghilangkan/extract email</li>
	<li>Menghilangkan/extract nomer telephone</li>
	<li>Menghilangkan/extract URL di string.</li>
	<li>Alpha Numeric filtering</li>
	<li>Wild Card Search</li>
	<li>Cleaning hashTags di Media Sosial</li>
</ol>


In [ ]:
# Extracting atau replacing eMail.
import re
emailPattern = re.compile(r'[\w._%+-]+@[\w\.-]+\.[a-zA-Z]{2,4}')

txt = 'Contact kami di admin@nlpindonesia.org, nlp.indonesia@sci.yahoo.co.id, atau nlp_nusantara@internet.net'

print( re.sub(emailPattern, ' ', txt) )# clean email
eMailS = re.findall( emailPattern, txt )
print( 'email yang ditemukan: ', str(eMailS) )

In [ ]:
# Pola telephone : \d penanda angka di reguler Expression, \s spasi, dan "|" adalah "atau"
# "?" menyatakan pilihan (optional): colou?r sesuai dengan colour atau color.

phonePattern = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
txt = 'Contact kami di 021-7634562 atau 021-763-4562 atau 021 763 4562 atau 0822959020 atau +628199258688'
print(re.sub(phonePattern,'***',txt))# clean phone
    
noTelp = re.findall(phonePattern,txt)
print('Nomer telephone yang ditemukan: ',str(noTelp))

In [ ]:
# Pola telephone 2: untuk setiap angka 8-14 digits dipisahkan oleh "spasi", "," atau "."
phonePattern = re.compile(r'\b\d{8,14}\b')
txt = 'Contact kami di 082295203040 atau +6282295203040'
print(re.sub(phonePattern,'***',txt))# clean phone
    
noTelp = re.findall(phonePattern,txt)

for no in noTelp:
    if no[0]!='0':
        print('+' + no)
    else:
        print(no)
#print('Nomer telephone yang ditemukan: ',str(noTelp))

In [ ]:
# Website URLS http(s) .... untuk ftp trivial
urlPattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

txt = 'website reguler expression & my site : https://www.regular-expressions.info/ & https://tau-data.id'
print(re.sub(urlPattern,' ',txt))# clean urls

URLs = re.findall(urlPattern,txt)# get URLs
print('URL yang ditemukan: ',str(URLs))

In [ ]:
import time

time.sleep(5)
'Done'

In [ ]:
# cleaning non alpha-numeric
txt = 'Hi! @Mukidi, apa kabar? #sapa_Pagi.'
print(re.sub(r'[^\w]',' ',txt))
# atau jika ingin exclude titik dan koma 
# re.sub(r'[^.,a-zA-Z0-9 \n\.]','',txt)

In [ ]:
# alternative 2:
print(''.join([t for t in txt if t.isalnum() or t==' ']))
# ada perbedaan?

In [ ]:
# Cleaning hashTags dalam posting media sosial
tweet = 'oh IoT, #AndaiSajaIaTahu #ApaYangAkuRasah... #AlayersTweet #d2d'

getHashtags = re.compile(r"#(\w+)")
print("Tags = {0}".format(re.findall(getHashtags, tweet)))
# temukan hanya tags ... perhatikan IoT bukan Tags walau ada huruf besar & kecil dalam satu kata

In [ ]:
pisahtags = re.compile(r'[A-Z][^A-Z]*')

for tags in re.findall(getHashtags, tweet):
    print(re.findall(pisahtags, tags))

In [ ]:
# Mengganti hashtags dengan kata dasar pembentuknya
tweet = 'oh IoT, #AndaiSajaIaTahu #ApaYangAkuRasah... #AlayersTweet'
tagS = re.findall(getHashtags, tweet)
for tag in tagS:
    proper_words = ' '.join(re.findall(pisahtags, tag))
    tweet = tweet.replace('#'+tag,proper_words)

print(tweet)

In [ ]:
s = 'The25XYZ3abc'
re.split('(\d+)',s)

## Encoding-Decoding:

<ul>
	<li>Hal berikutnya yang perlu diperhatikan dalam memproses data teks adalah encoding-decoding.</li>
	<li>Contoh Encoding: ASCII, utf, latin, dsb.</li>
	<li>saya membahas lebih jauh tetang encoding disini:&nbsp;<br />
	<a href="https://tau-data.id/memahami-string-python/" target="_blank">https://tau-data.id/memahami-string-python/</a></li>
	<li>Berikut adalah sebuah contoh sederhana tantangan proses encoding-decoding ketika kita hendak memproses data yang berasal dari internet atau media sosial.</li>
</ul>


In [ ]:
# kita bisa menggunakan modul unidecode untuk mendapatkan representasi ASCII terdekat
from unidecode import unidecode

T = "ḊḕḀṙ ₲ØĐ, p̾l̾e̾a̾s̾e ḧḕḶṖ ṁḕ"
print(unidecode(T).lower())
# Bahasa Indonesia dan Inggris secara umum mampu direpresentasikan dalam encoding ASCII: 
# https://en.wikipedia.org/wiki/ASCII

In [ ]:
# Kita juga bisa membersihkan posting media sosial/website dengan entitas html menggunakan fungsi "unescape" di modul "html"
from html import unescape

print(unescape('Satu &lt; Tiga&nbsp;&amp; &#169; adalah simbol Copyright'))

In [ ]:
def pungsi(x,y):
    z=x**y
    return z

pungsi(2,3)

<p><strong>Latihan 1</strong>:</p>

<p>Diberikan&nbsp;tweet berikut:<br />
<strong>tweet&nbsp;</strong>=&nbsp; &quot;<em><strong>The #OctopiPower is &amp;gt; Sharks! &amp;amp; they&#39;re awsm! So happy to see them here <a href="http://www.octopusVSshark.com" target="_blank">http://www.octopusVSshark.com</a> !</strong></em>&quot;</p>

<p>preprocess tweet diatas sehingga didapatkan tweet seperti ini (Gunakan sembarang modul yang mendukung):<br />
<strong>tweet</strong>= &quot;<em><strong>the octopus power is &gt; shark ! &amp; they are awesome ! so happy to see them here !</strong></em>&quot;</p>

<p><u><strong>Petunjuk/Hints</strong></u>:</p>

<ol>
	<li>Buat satu atau lebih fungsi untuk memudahkan, misal fungsi <em>fixTags </em>dan <em>cleanText</em>.</li>
	<li>fix kata &quot;<em>they&#39;re</em>&quot; dan <em>awsm </em>dengan teknik sederhana <em>dictionary fix</em>.</li>
	<li>Hati-hati terhadap <u><strong>urutan </strong></u>aksi di preprocessing karena akan mengakibatkan hasil yang berbeda.<br />
	Kelak di segmen berikutnya urutan aksi ini akan disebut sebagai &quot;<big><strong>Pipelining</strong></big>&quot;.</li>
	<li>Code solusi latihan ini dengan dasar fikiran bahwa solusinya nanti akan bisa digunakan untuk sembarang preprocessing.</li>
	<li>At this point, jangan hawatirkan dulu scalability/efisiensi.</li>
</ol>


In [ ]:
%time
# Coba jawaban Latihan [1] di cell ini: 
# Salah satu contoh jawaban latihan ini ada di cell paling bawah. 
# Namun coba untuk tidak melihat jawaban tersebut


<p><strong>Latihan 2</strong>:</p>

<p>Bagaimana caranya memfilter kata-kata (token) yang terdiri dari huruf dan angka (misal <em>b29nf, _24x_,&nbsp;</em>dsb)?</p>


In [ ]:
%time
# Coba jawaban Latihan [2] di cell ini: 
# Salah satu contoh jawaban latihan ini ada di cell paling bawah. 
# Namun coba untuk tidak melihat jawaban tersebut
T = 'pesawat b29 dan mig276 adalah kepunyaan fhg347x dan _24x_'
# Harapan jawaban T = 'pesawat dan adalah kepunyaan dan'
# Petunjuk: gunakan property "isalpha" pada variabel string di Python.

<h1>End of Module NLPTM-01</h1>
<hr />
